In [3]:
#IMPORTANT LIB
import streamlit as st
import sentencepiece
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain  
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import base64 
import importlib
import subprocess
import sys
from streamlit_jupyter import StreamlitPatcher, tqdm

In [4]:
#FILE LOADER AND PREPROCESSING
StreamlitPatcher().jupyter()
summarizer = pipeline("summarization", model="MBZUAI/LaMini-Flan-T5-248M")

# If you also need tokenizer and model explicitly:
tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")
model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")

def file_preprocessing(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    texts = text_splitter.split_documents(pages)
    final_text = ""
    for text in texts:
        print(text)
        final_text = final_text + text.page_content
    return final_text

Device set to use cpu


In [5]:
#LLM PIPELINE
def llm_pipeline(filepath):
    pipe_sum = pipeline(
        'summerization',
        model = model,
        tokernzier = tokenizer,
        max_length = 500,
        min_length = 50,
    )
    input_text = file_preprocessing(filepath)
    result = pipe_sum(input_text)
    result = result[0]['summary_text']
    return result

In [ ]:
#STREAMLIT CODE UI/UX
@st.cache_data
def display_PDF(file):
    with open(file, "rb") as f:
        base64_pdf = base64.b64decode(f.read().decode('utf-8'))

    pdf_display = F'<ifram src="data:application/pdf;base64,{base64_pdf}" width="100%" height="600" type="application/pdf"></iframe>'
    st.markdown(pdf_display, unsafe_allow_html=True)
st.set_page_config(layout='wide', page_title='Summerization App')

def main():
    st.title('Document Classification using LLM')
    uploaded_file = st.file_uploader("Upload your PDF", type=['pdf'])

    if uploaded_file is not None:
        if st.button("Summerize"):
            col1, col2 = st.columns(2)
            filepath = "Data_PDF/"+uploaded_file.name
            with open(filepath, 'wb') as temp_file:
                temp_file.write(uploaded_file.read())
            with col1:
                st.info("Uploaded PDF file")
                pdf_viewer = display_PDF(filepath)
            with col2:
                st.info("Summerization is below")    

                summary = llm_pipeline(filepath)
                st.success(summary)

2025-08-16 10:52:23.128 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-16 10:52:23.131 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
if __name__ == '__main__':
    main()

2025-08-16 10:52:23.152 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.349 
  command:

    streamlit run c:\Users\Asus\OneDrive\Desktop\Shaily\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-16 10:52:23.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 10:52:23.356 Thre